In [3]:
import torch 
from torch import nn
import numpy as np
import random as rnd
import matplotlib
import time
import copy as copy
# Set the backend BEFORE importing pyplot
matplotlib.use('Qt5Agg')  # Or 'Qt5Agg', 'WXAgg', etc.

import matplotlib.pyplot as plt


device = "cuda" if torch.cuda.is_available() else "cpu"
device

#device = "cpu"

#Tabuleiro = Board().to(device)

'cuda'

In [4]:
m = 6
n = 7

def rnd_N(n):
    return(int(rnd.random()*n))

def remove_from_tensor(Tensor, element):
    mask = Tensor != element
    Tensor_after_remove = Tensor[mask]
    return(Tensor_after_remove)



def Check_Conections(positionX, positionY, S): # positionY é de cima pra baixo
    List_Connections = [] # Formato conexão: [X, Y, n], n = 1,2,3,4 para leste, sudeste, sul, sudoeste
    if positionX < n-3:
        if S[positionY][positionX] == S[positionY][positionX+1] == S[positionY][positionX+2] == S[positionY][positionX+3]:
            List_Connections = List_Connections + [[positionX, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 1]])), dim=0)
    if positionX > 2:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX-2] == S[positionY][positionX-3]:
            List_Connections = List_Connections + [[positionX-3, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-3, positionY, 1]])), dim=0)
    if positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX] == S[positionY+2][positionX] == S[positionY+3][positionX]:
            List_Connections = List_Connections + [[positionX, positionY, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 3]])), dim=0)
    if positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY-2][positionX] == S[positionY-3][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-3, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-3, 3]])), dim=0)

    if positionX < n-3 and positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX+1] == S[positionY+2][positionX+2] == S[positionY+3][positionX+3]:
            List_Connections = List_Connections + [[positionX, positionY, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 2]])), dim=0)
    if positionX < n-3 and positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY-2][positionX+2] == S[positionY-3][positionX+3]:
            List_Connections = List_Connections + [[positionX+3, positionY-3, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+3, positionY-3, 4]])), dim=0)
    if positionX > 2 and positionY < m-3:
        if S[positionY][positionX] == S[positionY+1][positionX-1] == S[positionY+2][positionX-2] == S[positionY+3][positionX-3]:
            List_Connections = List_Connections + [[positionX, positionY, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY, 4]])), dim=0)
    if positionX > 2 and positionY > 2:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY-2][positionX-2] == S[positionY-3][positionX-3]:
            List_Connections = List_Connections + [[positionX-3, positionY-3, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-3, positionY-3, 2]])), dim=0)

    if n-2 > positionX > 0:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX+1] == S[positionY][positionX+2]:
            List_Connections = List_Connections + [[positionX-1, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-1, positionY, 1]])), dim=0)
    if m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY+1][positionX] == S[positionY+2][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-1, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-1, 3]])), dim=0)
    if n-2 > positionX > 0 and m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY+1][positionX+1] == S[positionY+2][positionX+2]:
            List_Connections = List_Connections + [[positionX-1, positionY-1, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-1, positionY-1, 2]])), dim=0)
    if n-1 > positionX > 1 and m-2 > positionY > 0:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY+1][positionX-1] == S[positionY+2][positionX-2]:
            List_Connections = List_Connections + [[positionX+1, positionY-1, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+1, positionY-1, 4]])), dim=0)

    if n-1 > positionX > 1:
        if S[positionY][positionX] == S[positionY][positionX-1] == S[positionY][positionX-2] == S[positionY][positionX+1]:
            List_Connections = List_Connections + [[positionX-2, positionY, 1]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-2, positionY, 1]])), dim=0)
    if m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX] == S[positionY-2][positionX] == S[positionY+1][positionX]:
            List_Connections = List_Connections + [[positionX, positionY-2, 3]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX, positionY-2, 3]])), dim=0)
    if n-1 > positionX > 1 and m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX-1] == S[positionY-2][positionX-2] == S[positionY+1][positionX+1]:
            List_Connections = List_Connections + [[positionX-2, positionY-2, 2]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX-2, positionY-2, 2]])), dim=0)
    if n-2 > positionX > 0 and m-1 > positionY > 1:
        if S[positionY][positionX] == S[positionY-1][positionX+1] == S[positionY-2][positionX+2] == S[positionY+1][positionX-1]:
            List_Connections = List_Connections + [[positionX+2, positionY-2, 4]]
            #List_Connections = torch.cat((List_Connections, torch.tensor([[positionX+2, positionY-2, 4]])), dim=0)

    return(List_Connections)

class Board:
    def __init__(self):
        self.board_state = torch.zeros(6, 7).tolist()
        self.available_moves = [0,1,2,3,4,5,6]
        self.List_Connections_1 = []
        self.List_Connections_2 = []
    '''
    def to(self, device):
        """Moves all internal tensors to the specified device."""
        self.board_state = self.board_state.to(device)
        self.available_moves = self.available_moves.to(device)
        self.List_Connections_1 = self.List_Connections_1.to(device)
        self.List_Connections_2 = self.List_Connections_2.to(device)
        # Return self to allow for chaining: obj = TensorContainer(...).to(device)
        return self
    '''
    def move(self, player, position):
        S = self.board_state
        A_M = self.available_moves
        LC1 = self.List_Connections_1
        LC2 = self.List_Connections_2
        if position not in A_M:
            print('position not available')
            L = [1]
            for i in range(2): # Isso aqui é só pra dar um erro quando o movimento é ilegal
                L[i] = 1
        else:
            # Atualizando o tabuleiro
            for i in range(6):
                if S[6-i-1][position] == 0:
                    if player == 1:
                        S[6-i-1][position] = 1
                        self.List_Connections_1 = self.List_Connections_1 + Check_Conections(position, 6-i-1, S) 
                    else:
                        S[6-i-1][position] = -1
                        self.List_Connections_2 = self.List_Connections_2 + Check_Conections(position, 6-i-1, S)
                    if i == 5: # Atualizando a lista de movimentos disponiveis
                        A_M.remove(position)
                    break
             
            
            self.board_state = S
            self.available_moves = A_M
                

In [46]:
'''          
Tabuleiro = Board()
print(Tabuleiro.board_state)
print(Tabuleiro.available_moves)
Terminou = False


NV1 = 0
NV2 = 0
NV0 = 0

LPV = []
Lx = []
for k in range(1):
    if k > 0 and k%100 == 0:
        print(k, NV1/(NV1+NV2+NV0), NV0/(NV1+NV2+NV0))
    
    
    Tabuleiro = Board()
    Terminou = False
    for i in range(7):
        for j in range(6):
            print(i, j)
            Jogadas_possiveis = Tabuleiro.available_moves
            if j%2 == 1:
                Tabuleiro.move(2, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])
            else:
                jogada = int(input('Jogada: '))
                Tabuleiro.move(1, jogada)
                Tabuleiro.move(1, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])
            print(Tabuleiro.board_state)
            print(Tabuleiro.available_moves)
            print(Tabuleiro.List_Connections_1)
            print(Tabuleiro.List_Connections_2)
            
            
            #print(Tabuleiro.board_state)
            #print(Tabuleiro.available_moves)
            #print(Tabuleiro.List_Connections_1)
            #print(Tabuleiro.List_Connections_2)
            
            if len(Tabuleiro.List_Connections_1) > 0:
                #print('vitória do jogador 1')
                NV1 = NV1 + 1
                Terminou = True
                break
            if len(Tabuleiro.List_Connections_2) > 0:
                #print('vitória do jogador 2')
                NV2 = NV2 + 1
                Terminou = True
                break
        if Terminou == True:
            LPV = LPV + [NV1/(NV1+NV2)]
            Lx = Lx + [k]
            break
    if Terminou == False:
        #print(Tabuleiro.board_state)
        NV0 = NV0 + 1'''

"          \nTabuleiro = Board()\nprint(Tabuleiro.board_state)\nprint(Tabuleiro.available_moves)\nTerminou = False\n\n\nNV1 = 0\nNV2 = 0\nNV0 = 0\n\nLPV = []\nLx = []\nfor k in range(1):\n    if k > 0 and k%100 == 0:\n        print(k, NV1/(NV1+NV2+NV0), NV0/(NV1+NV2+NV0))\n    \n    \n    Tabuleiro = Board()\n    Terminou = False\n    for i in range(7):\n        for j in range(6):\n            print(i, j)\n            Jogadas_possiveis = Tabuleiro.available_moves\n            if j%2 == 1:\n                Tabuleiro.move(2, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])\n            else:\n                jogada = int(input('Jogada: '))\n                Tabuleiro.move(1, jogada)\n                Tabuleiro.move(1, Jogadas_possiveis[rnd_N(len(Jogadas_possiveis))])\n            print(Tabuleiro.board_state)\n            print(Tabuleiro.available_moves)\n            print(Tabuleiro.List_Connections_1)\n            print(Tabuleiro.List_Connections_2)\n            \n            \n          

In [28]:
'''Depois a gente tem que ver quais tensores require grad e quais não, pra ficar mais eficiente'''
                
def minimax(Current_board_object, Model, depth, Player, alpha, beta): # Player = {1, != 1}
    Current_State = Current_board_object.board_state
    Current_Available_Moves = Current_board_object.available_moves
    if len(Current_board_object.List_Connections_1) > 0:
        return 1, None, True
    if len(Current_board_object.List_Connections_2) > 0:
        return 0, None, True
    elif len(Current_Available_Moves) == 0:
        return 0.5, None, True
    elif depth == 0:
        with torch.inference_mode():
            return Model(Board_to_input(torch.tensor(Current_State).to(device), torch.tensor([Player]).to(device))).item(), None, False

    else:
        if Player == 1: # Player 1 is the maximizer
            End_Node = True
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            Current_Evaluation = 0 # We start by assuming the worst possible case for player 1
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(1, move)
                Evaluation_of_child, a, b = minimax(child_board, Model, depth-1, 2, alpha, beta)
                if Evaluation_of_child == 1:
                    if b == True:
                        return 1, move, True
                    else:
                        return 1, move, False
                if Evaluation_of_child > Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                alpha = max(alpha, Current_Evaluation)
                if beta <= alpha:
                    break
        else:
            End_Node = True
            Current_Evaluation = 1 # We start by assuming the worst possible case for player 2
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(2, move)
                Evaluation_of_child,  a, b = minimax(child_board, Model, depth-1, 1, alpha, beta)
                if Evaluation_of_child == 0:
                    if b == True:
                        return 0, move, True
                    else:
                        return 0, move, False6
                if Evaluation_of_child < Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                beta = min(beta, Current_Evaluation)
                if beta <= alpha:
                    break
        
        
        return Current_Evaluation, best_move, End_Node 


def minimax_pretraining(Current_board_object, depth, Player, alpha, beta): # Player = {1, != 1}
    Current_State = Current_board_object.board_state
    Current_Available_Moves = Current_board_object.available_moves
    if len(Current_board_object.List_Connections_1) > 0:
        #print('A1')
        return 1, None, True
    if len(Current_board_object.List_Connections_2) > 0:
        #print('A2')
        return 0, None, True
    elif len(Current_Available_Moves) == 0:
        #print('A8')
        return 0.5, None, True
    elif depth == 0:
        #print('A9')
        return 0.5, None, False

    else:
        if Player == 1: # Player 1 is the maximizer
            End_Node = True
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            Current_Evaluation = 0 # We start by assuming the worst possible case for player 1
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(1, move)
                Evaluation_of_child, a, b = minimax_pretraining(child_board, depth-1, 2, alpha, beta)
                if Evaluation_of_child == 1:
                    if b == True:
                        #print('A3')
                        return 1, move, True
                    else:
                        #print('A4')
                        return 1, move, False
                if Evaluation_of_child > Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                alpha = max(alpha, Current_Evaluation)
                if beta <= alpha:
                    break
        else:
            End_Node = True
            Current_Evaluation = 1 # We start by assuming the worst possible case for player 2
            best_move = Current_Available_Moves[rnd_N(len(Current_Available_Moves))]
            
            for move in Current_Available_Moves:
                child_board = Board()
                child_board.board_state = copy.deepcopy(Current_State)
                child_board.available_moves = copy.deepcopy(Current_Available_Moves)
                child_board.move(2, move)
                Evaluation_of_child,  a, b = minimax_pretraining(child_board, depth-1, 1, alpha, beta)
                if Evaluation_of_child == 0:
                    if b == True:
                        #print('A5')
                        return 0, move, True
                    else:
                        #print('A6')
                        return 0, move, False
                if Evaluation_of_child < Current_Evaluation:
                    End_Node = False
                    Current_Evaluation = Evaluation_of_child
                    best_move = move
                beta = min(beta, Current_Evaluation)
                if beta <= alpha:
                    break
        
        #print('A7')
        return Current_Evaluation, best_move, End_Node

In [13]:
class Model_Connect_4(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=m*n+1, out_features=100)
        self.layer_2 = nn.Linear(in_features=100, out_features=100) 
        self.layer_3 = nn.Linear(in_features=100, out_features=100)
        self.layer_4 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return torch.squeeze(self.sigmoid(self.layer_4(self.relu(self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x))))))))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

class Model_Connect_4_layers3(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=m*n+1, out_features=100)
        self.layer_2 = nn.Linear(in_features=100, out_features=100) 
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return torch.squeeze(self.sigmoid(self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x))))))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2



In [27]:
def Baches_to_input(Batch_X, Batch_Players):
    size_of_batch = len(Batch_X)
    reshaped_X = torch.squeeze(Batch_X.reshape(size_of_batch, 1, -1))
    result = torch.cat((reshaped_X, Batch_Players), dim = 1)
    return result

def Board_to_input(state, player):
    reshaped_board = torch.squeeze(state.reshape(1, -1))
    result = torch.cat((reshaped_board, player), dim = 0)
    return result

In [37]:
def training_Batch(N, Base_model, depth, P_rnd):
    Batch_X = []
    Batch_Y = []
    Batch_Players = []
    for game in range(N):
        board = Board()
        if game%100 == 0:
            print('game:', game)
        for movement in range(m*n):
            A_M = board.available_moves
            Board_State = board.board_state

            if movement%2 == 0:
                player = 1
            else:
                player = 2
            Player_To_Neuron = -player*2 + 3
            
            State_to_append = copy.deepcopy(Board_State)

            Evaluation, best_move, End_node = minimax(board, Base_model, depth, player, 0, 1)
            if (End_node == True and rnd.random() > 0.6) or End_node == False:
                Batch_X.append(State_to_append)
                Batch_Y.append(Evaluation)
                Batch_Players.append(Player_To_Neuron)
                #print(torch.tensor(board.board_state))
                #print(Evaluation, best_move, End_node)

            if rnd.random() > P_rnd:
                board.move(player, best_move)
            else:
                board.move(player, A_M[rnd_N(len(A_M))])
            #print(torch.tensor(Board_State))
            #print(Evaluation, best_move)
            if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                break
            
        Batch_Players.append(Player_To_Neuron)
        Batch_X.append(copy.deepcopy(board.board_state))
        
        
        if len(board.List_Connections_2) > 0:
            Result = torch.tensor(0).to(device)
        elif len(board.List_Connections_1) > 0:
            Result = torch.tensor(1).to(device)
        else:
            Result = torch.tensor(0.5).to(device)
        Batch_Y.append(Result)

        
    #return Batch_X, Batch_Y         
    return torch.tensor(Batch_X).to(device), torch.tensor(Batch_Y).to(device), torch.unsqueeze(torch.tensor(Batch_Players).to(device), dim = -1)

def pretraining_Batch(N, depth, P_rnd):
    Batch_X = []
    Batch_Y = []
    Batch_Players = []
    for game in range(N):
        board = Board()
        if game%100 == 0:
            print('game:', game)
        for movement in range(m*n):
            A_M = board.available_moves
            Board_State = board.board_state

            if movement%2 == 0:
                player = 1
            else:
                player = 2

            Player_To_Neuron = -player*2 + 3

            
            State_to_append = copy.deepcopy(Board_State)

            Evaluation, best_move, End_node = minimax_pretraining(board, depth, player, 0, 1)
            if (Evaluation == 0.5 and rnd.random() > 0.6) or Evaluation != 0.5:
                Batch_X.append(State_to_append)
                Batch_Y.append(Evaluation) 
                Batch_Players.append(Player_To_Neuron)
            #print(torch.tensor(board.board_state))
            #print(Evaluation, best_move, End_node)

            if rnd.random() > P_rnd:
                board.move(player, best_move)
            else:
                board.move(player, A_M[rnd_N(len(A_M))])
            #print(torch.tensor(Board_State))
            #print(Evaluation, best_move)
            if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                break
            
        Batch_Players.append(Player_To_Neuron)
        Batch_X.append(copy.deepcopy(board.board_state))
        
        
        if len(board.List_Connections_2) > 0:
            Result = torch.tensor(0).to(device)
        elif len(board.List_Connections_1) > 0:
            Result = torch.tensor(1).to(device)
        else:
            Result = torch.tensor(0.5).to(device)
        Batch_Y.append(Result)

        
    #return Batch_X, Batch_Y         
    return torch.tensor(Batch_X).to(device), torch.tensor(Batch_Y).to(device), torch.unsqueeze(torch.tensor(Batch_Players).to(device), dim = -1)

In [35]:

Player_Model = Model_Connect_4().to(device)


In [38]:
def MPE(y_pred, Data_Y, power): # Calculates the mean of the error raised to a given power.

  error = y_pred - Data_Y
  loss = torch.mean(torch.pow(error, power))
  return loss



def train(Player_Model, Base_model, depth, Iterations, Epochs, Batch_Size, Print_Epoch_Loss):

    loss_function = nn.MSELoss()
    
    #optimizer = torch.optim.SGD(params = Player_Model.parameters(), lr = 1)
    optimizer = torch.optim.Adam(Player_Model.parameters(), lr=2e-3, betas=(0.9, 0.999))
    H=100
    
    
    
    for iteration in range(Iterations):
        if iteration%H == 0:
            print(iteration)
        t1 = time.time()
        training_data_X, training_data_Y, players = training_Batch(Batch_Size, Base_model, depth, 0.5)
        t2 = time.time()

        
        print('time games:', t2-t1)

        
        for epoch in range((1+iteration)*Epochs):
            
            y_pred = Player_Model(Baches_to_input(training_data_X, players).float())
           
            #loss = loss_function(y_pred, training_data_Y.float())       
            loss = 100*MPE(y_pred, training_data_Y.float(), 4)
            
            if epoch == 0 and iteration%H == 0: 
                print(loss)

            if epoch%Print_Epoch_Loss == 0:
                print(epoch, loss)
                
            if epoch == Epochs-1 and iteration%H == 0:
                print('      ', loss)
    
            optimizer.zero_grad()
        
            loss.backward()
        
            optimizer.step()
        
        t3 = time.time()
        print('time optimizing:', t3-t2)


    return(Player_Model)





def pretrain(Player_Model, depth, Iterations, Epochs, Batch_Size, Print_Epoch_Loss):

    loss_function = nn.MSELoss()
    
    #optimizer = torch.optim.SGD(params = Player_Model.parameters(), lr = 1)
    optimizer = torch.optim.Adam(Player_Model.parameters(), lr=2e-3, betas=(0.9, 0.999))
    H=100
    
    
    
    for iteration in range(Iterations):
        if iteration%H == 0:
            print(iteration)
        t1 = time.time()
        training_data_X, training_data_Y, players = pretraining_Batch(Batch_Size, depth, 0.5)
        t2 = time.time()

        
        print('time games:', t2-t1)

        
        for epoch in range(Epochs):
            
            y_pred = Player_Model(Baches_to_input(training_data_X, players).float())
           
            #loss = loss_function(y_pred, training_data_Y.float())       
            loss = 100*MPE(y_pred, training_data_Y.float(), 4)
            
            if epoch == 0 and iteration%H == 0: 
                print(loss)

            if epoch%Print_Epoch_Loss == 0:
                print(epoch, loss)
                
            if epoch == Epochs-1 and iteration%H == 0:
                print('      ', loss)
    
            optimizer.zero_grad()
        
            loss.backward()
        
            optimizer.step()
        
        t3 = time.time()
        print('time optimizing:', t3-t2)


    return(Player_Model)
        

In [40]:
Player_Model = pretrain(Player_Model, 3, 3, 5000, 10000, 100)
                    

0
game: 0
game: 100
game: 200
game: 300
game: 400
game: 500
game: 600
game: 700
game: 800
game: 900
game: 1000
game: 1100
game: 1200
game: 1300
game: 1400
game: 1500
game: 1600
game: 1700
game: 1800
game: 1900
game: 2000
game: 2100
game: 2200
game: 2300
game: 2400
game: 2500
game: 2600
game: 2700
game: 2800
game: 2900
time games: 135.88455247879028
tensor(3.7910, device='cuda:0', grad_fn=<MulBackward0>)
0 tensor(3.7910, device='cuda:0', grad_fn=<MulBackward0>)
100 tensor(0.3779, device='cuda:0', grad_fn=<MulBackward0>)
200 tensor(0.1562, device='cuda:0', grad_fn=<MulBackward0>)
300 tensor(0.1029, device='cuda:0', grad_fn=<MulBackward0>)
400 tensor(0.0756, device='cuda:0', grad_fn=<MulBackward0>)
       tensor(0.0600, device='cuda:0', grad_fn=<MulBackward0>)
time optimizing: 8.5688157081604
game: 0
game: 100
game: 200
game: 300
game: 400
game: 500
game: 600
game: 700
game: 800
game: 900
game: 1000
game: 1100
game: 1200
game: 1300
game: 1400
game: 1500
game: 1600
game: 1700
game: 1800
ga

In [42]:
'''Teste das previsões do modelo'''
def model_test(model, games, depth):
    training_data_X, training_data_Y, players = pretraining_Batch(games, depth, 0.5)
    y_pred = model(Baches_to_input(training_data_X, players).float())
    
    
    
    total05=0
    correct05=0
    total10=0
    correct10=0
    total00=0
    correct00=0
    correct_high=0
    correct_low=0
    for i in range(len(training_data_Y)):
        
        if training_data_Y[i].item() == 0.5:
            total05 = total05+1
            if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
                correct05  += 1
        if training_data_Y[i].item() == 1:
            total10 += 1
            if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
                correct10 += 1
            if y_pred[i].item() > 0.5:
                correct_high += 1
        if training_data_Y[i].item() == 0:
            total00 += 1
            if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
                correct00 += 1
            if y_pred[i].item() < 0.5:
                correct_low += 1

    return correct05/total05, correct10/total10, correct00/total00, (correct_high+correct_low)/(total10+total00)
'''
print('Accuracy 0.5:', correct05/total05)
print('Accuracy 1:', correct10/total10)
print('Accuracy 0:', correct00/total00)


print(torch.cat((training_data_Y.unsqueeze(-1), y_pred.unsqueeze(-1)), dim=1))

for i in range(len(training_data_Y)):
    print(training_data_X[i], training_data_Y[i])'''

"\nprint('Accuracy 0.5:', correct05/total05)\nprint('Accuracy 1:', correct10/total10)\nprint('Accuracy 0:', correct00/total00)\n\n\nprint(torch.cat((training_data_Y.unsqueeze(-1), y_pred.unsqueeze(-1)), dim=1))\n\nfor i in range(len(training_data_Y)):\n    print(training_data_X[i], training_data_Y[i])"

In [ ]:
print(model_test(Player_Model, 100, 5))

In [11]:
def play_against_model(Player_Model, depth, Human_player):
    New_game = True
    while New_game == True:
        New_game = input('New game? Y/N:')
        if New_game == 'y' or New_game == 'Y':
            New_game = True
        else:
            New_game = False
            break
            
        board = Board()
        #Human_player = rnd_N(2)+1
        for Turn in range(m*n):
            
            if Turn%2 == Human_player-1:               
                Valid_Move = False
                while Valid_Move == False:
                    Move = int(input('Your move:'))
                    if Move in board.available_moves:
                        Valid_Move = True
                    else:
                        print('Invalid move')
                board.move(Human_player, Move)
                print(torch.tensor(board.board_state))
                
                if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                    print('You won!')
                    break
            
            else:
                Evaluation, best_move, End_node = minimax(board, Player_Model, depth, Turn%2+1, 0, 1)
                print(Evaluation, "Model' move:", best_move)
                board.move(Turn%2+1, best_move)
                print(torch.tensor(board.board_state))

                if len(board.List_Connections_1) > 0 or len(board.List_Connections_2) > 0:
                    print('The model won (=> vc é mt burro)')
                    break



In [188]:
play_against_model(Player_Model, 5, 2)

New game? Y/N: y


0.5032333731651306 Model' move: 3
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.]])


Your move: 2


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.]])
0.5365138053894043 Model' move: 3
tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.]])


Your move: 3


...

Your move: 3


tensor([[ 0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0., -1.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  1., -1.,  0.,  0.,  0.],
        [ 1.,  1., -1.,  1.,  0., -1.,  0.],
        [-1., -1., -1.,  1., -1.,  1.,  0.]])
1 Model' move: 0
tensor([[ 0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0., -1.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1., -1.,  0.,  0.,  0.],
        [ 1.,  1., -1.,  1.,  0., -1.,  0.],
        [-1., -1., -1.,  1., -1.,  1.,  0.]])


Your move: 0


tensor([[ 0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0., -1.,  1., -1.,  0.,  0.,  0.],
        [-1.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1., -1.,  0.,  0.,  0.],
        [ 1.,  1., -1.,  1.,  0., -1.,  0.],
        [-1., -1., -1.,  1., -1.,  1.,  0.]])
1 Model' move: 0
tensor([[ 0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 1., -1.,  1., -1.,  0.,  0.,  0.],
        [-1.,  1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1., -1.,  0.,  0.,  0.],
        [ 1.,  1., -1.,  1.,  0., -1.,  0.],
        [-1., -1., -1.,  1., -1.,  1.,  0.]])
The model won (=> vc é mt burro)


KeyboardInterrupt: Interrupted by user

In [318]:
'''

PATH = "C4_pretrained_Model_v3.pth"
torch.save(Player_Model.state_dict(), PATH)'''

In [14]:
Player_Model = Model_Connect_4_().to(device)

Player_Model.load_state_dict(torch.load('C4_pretrained_Model_v2.pth'))

<All keys matched successfully>

In [47]:

'''
training_data_X, training_data_Y, players = pretraining_Batch(1000, 3, 0.5)
y_pred = Player_Model_2(training_data_X.float())


total05=0
correct05=0
total10=0
correct10=0
total00=0
correct00=0
correct_high=0
correct_low=0
for i in range(len(training_data_Y)):
    
    if training_data_Y[i].item() == 0.5:
        total05 = total05+1
        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
            correct05  += 1
    if training_data_Y[i].item() == 1:
        total10 += 1
        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
            correct10 += 1
        if y_pred[i].item() > 0.5:
                correct_high += 1
    if training_data_Y[i].item() == 0:
        total00 += 1
        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:
            correct00 += 1
        if y_pred[i].item() < 0.5:
                correct_low += 1


print('Accuracy 0.5:', correct05/total05)
print('Accuracy 1:', correct10/total10)
print('Accuracy 0:', correct00/total00)
print('Accuracy 1/0:', (correct_high+correct_low)/(total10+total00))'''

"\ntraining_data_X, training_data_Y, players = pretraining_Batch(1000, 3, 0.5)\ny_pred = Player_Model_2(training_data_X.float())\n\n\ntotal05=0\ncorrect05=0\ntotal10=0\ncorrect10=0\ntotal00=0\ncorrect00=0\ncorrect_high=0\ncorrect_low=0\nfor i in range(len(training_data_Y)):\n    \n    if training_data_Y[i].item() == 0.5:\n        total05 = total05+1\n        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:\n            correct05  += 1\n    if training_data_Y[i].item() == 1:\n        total10 += 1\n        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:\n            correct10 += 1\n        if y_pred[i].item() > 0.5:\n                correct_high += 1\n    if training_data_Y[i].item() == 0:\n        total00 += 1\n        if abs(training_data_Y[i].item() - y_pred[i].item()) < 0.25:\n            correct00 += 1\n        if y_pred[i].item() < 0.5:\n                correct_low += 1\n\n\nprint('Accuracy 0.5:', correct05/total05)\nprint('Accuracy 1:', correct10/total10)\